# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [61]:
# importar librerias que usamos en los distintos dias

from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [67]:
#check Open IA API
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [63]:
# constants

MODEL_GPT = 'gpt-4o-mini'
openai = OpenAI()
MODEL_LLAMA = 'llama3.2'

In [64]:
# Defino pregunta y system prompt. El user promt sera igual a la pregunta

#question = """
#Por favor explicame que hace este código
#yield from {book.get("author") for book in books if book.get("author")}
#"""

question = """
Por favor explicame que hace este código
display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

"""

system_prompt="Eres un experto en programación y análisis de código. Responde fácil, apto para no expertos y con precisión, por favor pon los resultados con salto de línea."

In [65]:
# Funcion para responder pregunta solo con markdown
def responder_pregunta(question, system_prompt):
    response =openai.chat.completions.create(
        model=MODEL_GPT, 
        messages=[
            {"role": "system", "content": system_prompt},  #igualo al user promt
            {"role": "user", "content": question}          # igualo el user promt a la variable question
        ]
        )
    result=response.choices[0].message.content
    return display(Markdown(result))
    
    

In [66]:
responder_pregunta(question,system_prompt)

Claro, te explicaré paso a paso qué hace este código:

1. **Inicialización del display:**
   ```python
   display_handle = display(Markdown(""), display_id=True)
   ```
   Aquí se está creando un objeto de visualización que mostrará texto en formato Markdown. Se inicia con un texto vacío y se asigna un identificador para que se pueda actualizar más tarde.

2. **Bucle sobre un flujo de datos:**
   ```python
   for chunk in stream:
   ```
   Este bucle itera sobre un conjunto de datos llamado `stream`. Cada `chunk` representa un pedazo de información que se recibe uno a la vez.

3. **Construcción de la respuesta:**
   ```python
   response += chunk.choices[0].delta.content or ''
   ```
   Aquí se está acumulando el contenido del primer fragmento (`choices[0].delta.content`) del `chunk` en la variable `response`. Si este contenido es `None` o vacío, se añade una cadena vacía (`''`).

4. **Limpieza del texto:**
   ```python
   response = response.replace("```", "").replace("markdown", "")
   ```
   En esta línea, se reemplazan todas las ocurrencias de ```` ``` ```` y la palabra `markdown` en la variable `response` con cadenas vacías, eliminándolas del texto que se va a mostrar.

5. **Actualización de la visualización:**
   ```python
   update_display(Markdown(response), display_id=display_handle.display_id)
   ```
   Finalmente, esta línea actualiza la visualización en la interfaz, mostrando el contenido que ahora está en `response`. Se utiliza el identificador `display_handle.display_id` para asegurarse de que se actualice la misma visualización que se creó al inicio.

En resumen, este código muestra en la interfaz un texto que se va construyendo en tiempo real a partir de un flujo de datos, eliminando ciertas partes innecesarias antes de mostrarlo.

In [53]:
# Funcion para responder pregunta solo con streaming
def stream_pregunta(question, system_prompt):
    stream =openai.chat.completions.create( #aca en lugar de utilizar responde utilizo stream
        model=MODEL_GPT, 
        messages=[
            {"role": "system", "content": system_prompt},  #igualo al user promt
            {"role": "user", "content": question}          # igualo el user promt a la variable question
        ],
        stream=True
        )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [54]:
stream_pregunta(question,system_prompt)

Este código está diseñado para mostrar de forma dinámica el contenido que se va generando en tiempo real (generalmente a partir de una conversación con un modelo de lenguaje) en un entorno que permite la visualización, como un cuaderno de Jupyter.

Aquí te explico cada parte:

1. **`display_handle = display(Markdown(""), display_id=True)`**:
   - Inicializa una visualización vacía en formato Markdown, dándole un identificador (ID) para poder actualizarla después. 
   - `display` es una función que muestra contenido en el cuaderno.
   - `Markdown("")` crea un bloque vacío que se verá como texto formateado.

2. **`for chunk in stream:`**:
   - Inicia un bucle que recorrerá cada "trozo" (chunk) de datos que llega desde una fuente llamada `stream`. Esto puede ser, por ejemplo, respuestas generadas en tiempo real de un modelo de lenguaje.

3. **`response += chunk.choices[0].delta.content or ''`**:
   - Aquí se va acumulando el contenido en la variable `response`. 
   - `chunk.choices[0].delta.content` accede al contenido del primer elemento de `choices` en el `chunk` actual. Si este contenido es `None`, se añade una cadena vacía ('') para no afectar la acumulación.

4. **`response = response.replace("", "").replace("", "")`**:
   - Se eliminan ciertos caracteres o palabras del texto que se ha acumulado en `response`, específicamente:
     - Se remueven "" (que suelen usarse para bloques de código en Markdown).
     - Se remueve la palabra "".

5. **`update_display(Markdown(response), display_id=display_handle.display_id)`**:
   - Finalmente, actualiza el contenido mostrado en la visualización inicial utilizando el nuevo `response` ya procesado.
   - Esto se hace utilizando `update_display`, que cambia el contenido del bloque Markdown sin crear uno nuevo, gracias al `display_id` que se guardó anteriormente.

Entonces, en resumen, el código permite mostrar y actualizar dinámicamente un texto que se genera a partir de un flujo de datos, eliminando ciertas partes no deseadas mientras se acumula el contenido.

In [58]:
# Utilizar Llama 3.2
OLLAMA_API = "http://localhost:11434/api/chat" ## conexion con el modelo
HEADERS = {"Content-Type": "application/json"} ## recojer las respuestas correctamente

import ollama

In [59]:
# Funcion para responder pregunta solo con markdown
def responder_pregunta_ollama(question, system_prompt):
    response =ollama.chat(
        model=MODEL_LLAMA, 
        messages=[
            {"role": "system", "content": system_prompt},  #igualo al user promt
            {"role": "user", "content": question}          # igualo el user promt a la variable question
        ]
        )
    result=response['message']['content']
    return display(Markdown(result))
    

In [60]:
responder_pregunta_ollama(question, system_prompt)

Este código parece ser parte de una aplicación que permite editar y visualizar contenido Markdown utilizando un editor de código. A continuación, te explicaré qué hace cada línea:

**1. `display_handle = display(Markdown(""), display_id=True)`**

*   Crea un objeto `Markdown` vacío (`""`).
*   Llama al método `display()` con el objeto `Markdown` creado anteriormente y establece la propiedad `display_id` en `True`. El resultado se almacena en la variable `display_handle`.

**2. `for chunk in stream:`**

*   Inicia un bucle de iteración (`for`) que itera sobre cada elemento (`chunk`) en el objeto `stream`. Sin embargo, no tengo información específica sobre qué es el contenido de `stream`, por lo que asumiré que se trata de una secuencia de datos que se van enviendo en forma de paquetes.

**3. `response += chunk.choices[0].delta.content or ''`**

*   Agrega el contenido (`content`) del primer elemento seleccionado (`choices[0]`) dentro del objeto delta (`delta`) a la variable `response`. La expresión `or ''` es un operador lógico "o" que permite unir una cadena vacía si la cadena resultante no es falsa.

**4. `response = response.replace("```","").replace("markdown", "")`**

*   Reemplaza dos cadenas de caracteres dentro de la variable `response`: `"```"` y `"markdown"`. La primera reemplaza cualquier código HTML (código entre tripletes de backticks) por nada, mientras que la segunda elimina el texto "markdown".

**5. `update_display(Markdown(response), display_id=display_handle.display_id)`**

*   Llama al método `update_display()` con un nuevo objeto `Markdown` creado a partir de la variable `response`, y establece la propiedad `display_id` en la misma dirección de la que se recibió el objeto original (`display_handle.display_id`). Esta llamada actualiza visualmente el contenido del editor.

En resumen, este código parece ser parte de una aplicación que permite editar y visualizar contenido Markdown. El ciclo principal itera sobre cada paquete de datos que llega desde `stream`, agrega el texto seleccionado al contenido del objeto Markdown (`response`), lo elimina de cualquier código HTML y actualiza visualmente el contenido en la pantalla.